In [1]:
import pandas as pd
import numpy as np
import pickle
import random
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from scipy.stats import spearmanr
from scipy.stats import kendalltau
import tqdm
import math
import warnings
warnings.filterwarnings('ignore')

### 1.Читаю данные и готовлю

In [2]:
players = pd.DataFrame.from_dict(pd.read_pickle('players.pkl')).T
tournaments = pd.DataFrame.from_dict(pd.read_pickle('tournaments.pkl')).T

In [3]:
dict_of_result = pd.read_pickle('results.pkl')

In [4]:
tournaments['dateStart'] = tournaments['dateStart'].apply(lambda x: x[:4])

In [5]:
train_id_tourn = tournaments[tournaments['dateStart'] == '2019']['id'].tolist()
test_id_tourn = tournaments[tournaments['dateStart'] == '2020']['id'].tolist()

In [6]:
def create_train(results, train_id):
    res = []
    for idx in train_id:
        result = results[idx]
        for team in result:
            if 'mask' in team and team['mask'] is not None and 'teamMembers' in team:
                for team_members in team['teamMembers']:
                    for id_quest, ball in enumerate(team['mask']):
                        if ball.isdigit():                   
                            question_id = str(idx) + ' ' + str(id_quest)
                            res.append([idx, team_members['player']['id'], question_id, int(ball)])
    random.shuffle(res)
    return pd.DataFrame(res, columns = ['tour_id', 'member_id', 'quest_id', 'results'])

In [7]:
train = create_train(dict_of_result, train_id_tourn)

In [8]:
def create_test(results, test_id):
    res = []
    for idx in test_id:
        result = results[idx]
        teams = []
        members = []
        for team in result:
            teams.append(team['team']['id'])
            members.append([])
            for member in team['teamMembers']:
                members[-1].append(member['player']['id'])
        res.append([idx, teams, members])
            
    return pd.DataFrame(res, columns = ['tour_id', 'teams_id', 'members_id'])

Выбрасываю все плохие записи о турнирах(если нет игроков, если длины не совпадают итд)

In [9]:
test = create_test(dict_of_result, test_id_tourn)
test = test[test.teams_id.apply(lambda x: len(x))>0]
test = test[test.members_id.apply(lambda x: len(x))>0]
test = test[test.teams_id.apply(lambda x: len(x)) == test.members_id.apply(lambda x: len(x))]
member_id = set(list(train.member_id.unique()))
bad_rows = set()
test = test.reset_index(drop = True)
for idx, row in enumerate(test.members_id.to_list()):
    for team in row:
        if len(team) < 1:
            bad_rows.add(idx)
        for memb in team:
            if memb not in member_id:
                bad_rows.add(idx)
print(bad_rows)
test = test[~test.index.isin(list(bad_rows))]
test

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 82, 83, 84, 85, 86, 87, 88, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 179, 180, 181, 182, 184, 187, 188, 189, 190, 191, 192, 193}


,tour_id,teams_id,members_id
19,5753,"[48583, 49225, 4032, 74621, 4252, 57024, 4622,...","[[72211, 10695, 40939, 76264, 36844], [57032, ..."
20,5754,"[65510, 50032, 62868, 48052, 75600, 4989, 7465...","[[3088, 52047, 1599, 23486], [23737, 28689, 20..."
31,5823,"[5170, 2855, 2826, 60039, 42100, 718, 28369, 6...","[[8915, 31980, 35604, 20864, 31981, 16589], [8..."
33,5848,"[47075, 72609, 670, 5455, 51739, 55867, 3875, ...","[[38458, 56000, 42511, 5990, 62252, 39596], [1..."
34,5849,"[49804, 48052, 51514, 4109, 55867, 47075, 5018...","[[30152, 27822, 28751, 30270, 27403, 4270], [6..."
55,6045,"[56211, 59419, 49146, 60977, 76986, 69703]","[[137496, 87809, 139390, 72302, 60898, 61948, ..."
80,6162,"[54799, 65697, 48762, 54238, 53877, 76705, 539...","[[149723, 131911, 131910, 147450, 149724, 1319..."
81,6163,"[76498, 48303, 60015, 65787, 55270, 48278, 623...","[[24850, 39431, 79987, 102108, 77954, 180753, ..."
89,6181,"[4795, 227, 49419, 40247, 318, 27161, 317, 553...","[[59103, 60252, 66121, 163947, 37518, 83335], ..."
90,6182,"[5379, 54414, 40247, 55270, 5534, 66744, 47425...","[[7361, 152977, 175002], [130283, 66122, 16568..."


In [10]:
test_id_tourn = test.tour_id.tolist()

в каждой строчке id турнира, id участника, id вопроса, и результат ответа команды

In [11]:
train.head()

,tour_id,member_id,quest_id,results
0,5699,210505,5699 28,0
1,6254,209805,6254 37,0
2,5390,61631,5390 32,0
3,5760,193660,5760 30,1
4,5760,40998,5760 38,0


### 2. Обучение бейзлайн модели

В задании сказано учесть сложность вопросов, это можно было сделать по-разному. Но я решил сделать onehot представление этих вопросов, так как это самый простой способ и в дальнейшем очень просто считать $ln\frac{\pi_0}{\pi_1}$ для ем алгоритма

In [12]:
enc = OneHotEncoder()
X = enc.fit_transform(np.array(train[['member_id', 'quest_id']]))
y = np.array(train['results'])

In [13]:
baseline = LogisticRegression()
baseline.fit(X, y)

LogisticRegression()

In [14]:
baseline.fit(X, y)

LogisticRegression()

Словарь для поиска по user_id его веса в лог реге

In [15]:
userid_to_idx = dict()
for idx, cat in enumerate(enc.categories_[0]):
    userid_to_idx[cat] = idx

In [16]:
coef_regr = baseline.coef_.copy()[0]

### 3. Оцениваем результат

Оценивание силы команды - среднее по весам игроков в получившейся модели.

In [17]:
def power_of_team(members_lst, coef_regr, userid_to_idx):
    power = 0
    for member in members_lst:
        if member in userid_to_idx:
            power += coef_regr[userid_to_idx[member]]
        else:
            power+= random.choice(coef_regr)
    return power / len(members_lst)

Функция для подсчёта средней корреляции на тестовом множестве.

In [18]:
def calc_corrrlation(tst_table, coefs, enc_dict):
    spear = []
    cand = []
    for i in range(tst_table.shape[0]):
        row = tst_table.iloc[i]
        ev = []
        n_teams = len(row['teams_id'])
        for idx, team in enumerate(row['teams_id']):
            ev.append([idx, power_of_team(row['members_id'][idx], coefs, enc_dict)])
        ev = sorted(ev, key=lambda x: x[1], reverse=True)
        for j in range(n_teams):
            ev[j]=ev[j][0]
        cor=spearmanr(list(range(n_teams)), ev)[0]
        if not math.isnan(cor):
            spear.append(cor)
        cor=kendalltau(list(range(n_teams)), ev)[0]
        if not math.isnan(cor):
            cand.append(cor)
    return np.array(spear).mean(), np.array(cand).mean()

In [19]:
baseline_corr = calc_corrrlation(test, coef_regr, userid_to_idx)
baseline_corr

(0.7452147748195809, 0.621599549901412)

Качество baseline модели

### 4 EM-алгоритм

Поставим задачу у нас отвечает на вопрос команда, но непонятно какой из участников конкретно знал вопрос, поэтому мы стремимся скорректировать единички в нашем датасете.<br>
Введем обозначения: <br>
y = 1 значит команда ответила на вопрос. y = 0, то никто в команда не знает ответ на вопрос.<br>
z = 1 значит игрок ответил на вопрос <br>
s = 1 значит вопрос попался <br>
По сути правильно предсказывать $p(z=1|\overline{x})$, а не $p(y=1|\overline{x})$ как было сделано в baseline модели. <br>
Наша задача аналогична задаче про сусликов на лекции<br>
$p(y=0|\overline{x},s=1) = p(y=0|z=1,s=1)*p(z=1|\overline{x},s=1) + p(y=0|z=0,s=1)*p(z=0|\overline{x},s=1)$ <br>
Первое слагаемое в сумме равно 0, так как $p(y=0|z=1,s=1) = 0$ то есть если игрок знает ответ, он бы ответил<br>
Получаем $p(y=0|\overline{x},s=1) = \frac{p(y=0,z=0|s=1)}{p(z=0|s=1)}p(z=0|\overline{x},s=1)$ <br>
Если положить что $n_n$ - вопросы, на которые игрок не ответил, а $n_u$ - мы не знаем исход , то придем к выражению 
$p(y=0|\overline{x},s=1) = \frac{n_n}{n_n+(1-\pi)*n_u}*p(z=0|\overline{x},s=1)$, где $\pi$ - это $p(z=1)$ <br>
По итогу ситуация получается как на лекции, только с другими поправками. <br>
$n_n$ и $n_u$ - я считал для каждого игрока отдельно, поправку $ln\frac{\pi_0}{\pi_1}$ прибавлял отдельно для каждого игрока.
Но опыты показали, что без поправки модель работает лучше(выше метрика). Поэтому я занулил эту поправку. Возможно неверно была определена $\pi$, но искать ее не было возможности.

In [24]:
class EM_algo:
    def __init__(self, X, y, userid_to_idx, tst_table, n_steps=4):
        self.X = X
        self.y = y
        self.steps = n_steps
        self.userid_to_idx = userid_to_idx
        self.tst_table = tst_table
        self.pi = 0.5
        self.coef_regr = None

    def power_of_team(self, members_lst):
        power = 0
        for member in members_lst:
            if member in self.userid_to_idx:
                power += self.coef_regr[self.userid_to_idx[member]]
            else:
                power += random.choice(coef_regr)
        return power / len(members_lst)

    def calc_correlation(self):
        self.coef_regr = self.model.coef_.copy()
        spear = []
        cand = []
        for i in range(self.tst_table.shape[0]):
            row = self.tst_table.iloc[i]
            ev = []
            n_teams = len(row['teams_id'])
            for idx, team in enumerate(row['teams_id']):
                ev.append([idx, self.power_of_team(row['members_id'][idx])])
            ev = sorted(ev, key=lambda x: x[1], reverse=True)
            for j in range(n_teams):
                ev[j] = ev[j][0]
            cor = spearmanr(list(range(n_teams)), ev)[0]
            if not math.isnan(cor):
                spear.append(cor)
            cor = kendalltau(list(range(n_teams)), ev)[0]
            if not math.isnan(cor):
                cand.append(cor)
        return np.array(spear).mean(), np.array(cand).mean()

    def train_baseline(self):
        self.model = LinearRegression()
        self.model.fit(X, y)


    def E_step(self):
        z = self.model.predict(self.X)
        z = (1 + np.exp(-z))**(-1)
        z = z * y
        return z
    
    def M_step(self, z):
        lnp0p1 = 0
        z[z==0] = 0.01
        target = np.log(z / (1 - z)) + lnp0p1
        self.model = LinearRegression()
        self.model.fit(X, target)
    
    def trainEM(self):
        self.train_baseline()
        for i in range(self.steps):
            z = self.E_step()
            self.M_step(z)

In [25]:
EM = EM_algo(X, y, userid_to_idx, test)
EM.trainEM()
EM_correlation = EM.calc_correlation()

In [26]:
EM_correlation

(0.7655967906224457, 0.6524269595588512)

In [29]:
print(f'Улучшение корреляции спирмана по сравнению с бейзлайном\t{EM_correlation[0] - baseline_corr[0]}')
print(f'Улучшение корреляции спирмана по сравнению с бейзлайном\t{EM_correlation[1] - baseline_corr[1]}')

Улучшение корреляции спирмана по сравнению с бейзлайном	0.020382015802864806
Улучшение корреляции спирмана по сравнению с бейзлайном	0.030827409657439242


Видно, что ЕМ алгоритм улучшает целевые показатели, однако из-за долгих расчетов, у меня нет возможности посмотреть предел для данного алгоритма